In [1]:
import scanpy as sc
import numpy as np
import functools
import jax
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 3

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [5]:
adata_ood.obs.head()

,sample,Size_Factor,n.umi,RT_well,Drug1,Drug2,Well,n_genes,n_genes_by_counts,total_counts,...,split,control,cell_type,cell_line,smiles_drug_1,smiles_drug_2,ood_1,ood_2,ood_3,ood_4
Cell,,,,,,,,,,,,,,,,,,,,,
A01_A02_RT_BC_22_Lig_BC_17,sciPlex_theis,0.899999,2416,RT_22,control,Dacinostat,B10,1572,1569,2412.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_26,sciPlex_theis,0.470861,1264,RT_22,control,Dacinostat,B10,902,901,1263.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_27,sciPlex_theis,0.508485,1365,RT_22,control,Dacinostat,B10,934,934,1365.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_64,sciPlex_theis,1.300827,3492,RT_22,control,Dacinostat,B10,2104,2102,3490.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood
A01_A02_RT_BC_22_Lig_BC_84,sciPlex_theis,0.790852,2123,RT_22,control,Dacinostat,B10,1434,1432,2121.0,...,ood,0,A549,A549,NaN,C1=CC=C2C(=C1)C(=CN2)CCN(CCO)CC3=CC=C(C=C3)/C=...,not ood,not ood,control+Dacinostat,not ood


In [6]:
adata_ood.obs["condition"].cat.categories

Index(['Panobinostat+Crizotinib', 'Panobinostat+Curcumin',
       'Panobinostat+SRT1720', 'Panobinostat+Sorafenib',
       'SRT2104+Alvespimycin', 'control', 'control+Alvespimycin',
       'control+Dacinostat'],
      dtype='object')

In [7]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Cediranib+PCI-34051',
       'Dacinostat+Danusertib', 'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051',
       'Givinostat+Carmofur', 'Givinostat+Cediranib', 'Givinostat+Crizotinib',
       'Givinostat+Curcumin', 'Givinostat+Dasatinib', 'Givinostat+SRT1720',
       'Givinostat+SRT2104', 'Givinostat+Sorafenib', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT2104',
       'Panobinostat+SRT3025', 'SRT3025+Cediranib', 'control',
       'control+Dasatinib', 'control+Givinostat', 'control+Panobinostat',
       'control+SRT2104'],
      dtype='object')

In [ ]:
Index(['Alvespimycin+Pirarubicin', 'Cediranib+PCI-34051',
       'Dacinostat+Danusertib', 'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051',
       'Givinostat+Carmofur', 'Givinostat+Cediranib', 'Givinostat+Crizotinib',
       'Givinostat+Curcumin', 'Givinostat+Dasatinib', 'Givinostat+SRT1720',
       'Givinostat+SRT2104', 'Givinostat+Sorafenib', 'Givinostat+Tanespimycin',
       'Panobinostat+Alvespimycin', 'Panobinostat+Dasatinib',
       'Panobinostat+PCI-34051', 'Panobinostat+SRT2104',
       'Panobinostat+SRT3025', 'SRT3025+Cediranib', 'control',
       'control+Dasatinib', 'control+Givinostat', 'control+Panobinostat',
       'control+SRT2104'],
      dtype='object')

In [10]:
control = adata_ood[adata_ood.obs["condition"]=="control"].X.toarray()
rng = np.random.default_rng(0)
drug_effect = {}
for cond in adata_train.obs["condition"].cat.categories:
    drug_effect[cond] = rng.choice(adata_train[adata_train.obs["condition"]==cond].X.toarray(), 500) - control

drug_effect["control"] = control # note this is not a displacement vector, but the control vector
    

In [11]:
drug_effect_ood = {}

drug_effect_ood['Panobinostat+Crizotinib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Crizotinib'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+Curcumin'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Curcumin'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+SRT1720'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+SRT1720'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['Panobinostat+Sorafenib'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+Sorafenib'] 
    - drug_effect['control+Givinostat']
    + drug_effect['control+Panobinostat']
)

drug_effect_ood['SRT2104+Alvespimycin'] = (
    drug_effect['control'] 
    + drug_effect['Givinostat+SRT2104'] 
    - drug_effect['control+Givinostat']
    + drug_effect['Panobinostat+Alvespimycin']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Alvespimycin'] = (
    drug_effect['control'] 
    + drug_effect['Panobinostat+Alvespimycin']
    - drug_effect['control+Panobinostat']
)

drug_effect_ood['control+Dacinostat'] = (
    drug_effect['control'] 
    + drug_effect['Dacinostat+Dasatinib']
    - drug_effect['control+Dasatinib']
)


In [12]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [14]:
import anndata as ad
import pandas as pd
all_data = []
conditions = []

for condition, array in drug_effect_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred_ood = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [15]:
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [16]:

ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [17]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)


In [18]:
adata_pred_ood

AnnData object with n_obs × n_vars = 3500 × 2000
    obs: 'condition'
    obsm: 'X_pca'

In [19]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")


In [20]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8693430253437587,
 'deg_ood_sinkhorn_div_1': 34.66800308227539,
 'deg_ood_sinkhorn_div_10': 12.288872855050224,
 'deg_ood_sinkhorn_div_100': 4.3400421142578125,
 'deg_ood_e_distance': 8.112258587078225,
 'deg_ood_mmd': 0.036582838743925095}

In [21]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [22]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8538484147616795,
 'decoded_ood_sinkhorn_div_1': 339.42017037527904,
 'decoded_ood_sinkhorn_div_10': 305.2273385184152,
 'decoded_ood_sinkhorn_div_100': 49.54132516043527,
 'decoded_ood_e_distance': 26.084349006545008,
 'decoded_ood_mmd': 0.061386519244738987}

In [23]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.4490099719592503,
 'encoded_ood_sinkhorn_div_1': 16.896723747253418,
 'encoded_ood_sinkhorn_div_10': 9.90711511884417,
 'encoded_ood_sinkhorn_div_100': 5.984116690499442,
 'encoded_ood_e_distance': 10.831397430465655,
 'encoded_ood_mmd': 0.05794300511479378}

In [24]:
ood_metrics_encoded

{'Panobinostat+Crizotinib': {'r_squared': 0.32966846227645874,
  'sinkhorn_div_1': 15.070901870727539,
  'sinkhorn_div_10': 8.501203536987305,
  'sinkhorn_div_100': 5.504108428955078,
  'e_distance': 10.203294412195568,
  'mmd': 0.040468946},
 'Panobinostat+Curcumin': {'r_squared': 0.7438454031944275,
  'sinkhorn_div_1': 10.154376029968262,
  'sinkhorn_div_10': 4.027549743652344,
  'sinkhorn_div_100': 1.0210227966308594,
  'e_distance': 1.1544617762563476,
  'mmd': 0.02124002},
 'Panobinostat+SRT1720': {'r_squared': 0.619867205619812,
  'sinkhorn_div_1': 9.07469367980957,
  'sinkhorn_div_10': 3.244497299194336,
  'sinkhorn_div_100': 0.6460456848144531,
  'e_distance': 0.5587659438080763,
  'mmd': 0.019316675},
 'Panobinostat+Sorafenib': {'r_squared': -0.49348628520965576,
  'sinkhorn_div_1': 13.365455627441406,
  'sinkhorn_div_10': 7.638601303100586,
  'sinkhorn_div_100': 5.724079132080078,
  'e_distance': 11.002868781102343,
  'mmd': 0.040227428},
 'SRT2104+Alvespimycin': {'r_squared'

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/additive_model"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
